In [1]:
!pip install pulp

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   -- ------------------------------------- 0.8/16.4 MB 26.4 MB/s eta 0:00:01
   --------- ------------------------------ 3.7/16.4 MB 39.8 MB/s eta 0:00:01
   --------------- ------------------------ 6.3/16.4 MB 40.4 MB/s eta 0:00:01
   ----------------------- ---------------- 9.6/16.4 MB 43.6 MB/s eta 0:00:01
   -------------------------------- ------- 13.2/16.4 MB 50.4 MB/s eta 0:00:01
   ------------------------------------- -- 15.5/16.4 MB 54.7 MB/s eta 0:00:01
   ---------------------------------------- 16.4/16.4 MB 50.4 MB/s eta 0:00:00


In [34]:
#part1
import re
import pulp

def parse_machine_line(line):
    # find brackets
    m = re.search(r'\[([.#]+)\]', line)
    pattern = m.group(1).strip()
    n = len(pattern)
    target_bits = [1 if ch == '#' else 0 for ch in pattern]
    # ignore joltage
    line_no_brace = re.sub(r'\{.*\}\s*$', '', line)
    # find button groups
    groups = re.findall(r'\(([0-9,\s]*)\)', line_no_brace)
    buttons = []
    for g in groups:
        g = g.strip()
        if g == '':
            indices = []
        else:
            indices = [int(x) for x in g.split(',') if x.strip() != '']
        vec = [0]*n
        for idx in indices:
            vec[idx] = 1
        buttons.append(vec)
    return target_bits, buttons

def solve_machine_ilp(target_bits, buttons):
    num_buttons = len(buttons)
    num_lights = len(target_bits)
    prob = pulp.LpProblem("Min_Button_Presses_mod2", pulp.LpMinimize)
    # binary variable
    x = [pulp.LpVariable(f"x_{j}", cat="Binary") for j in range(num_buttons)]
    # integer variables
    y = [pulp.LpVariable(f"y_{i}", lowBound=0, cat="Integer") for i in range(num_lights)]
    # minimize total presses
    prob += pulp.lpSum(x), "Minimize_total_presses"
    for i in range(num_lights):
        left = pulp.lpSum(buttons[j][i] * x[j] for j in range(num_buttons))
        prob += (left - 2*y[i] == target_bits[i]), f"LightParity_{i}"

    prob.solve(pulp.PULP_CBC_CMD())
    return sum(int(pulp.value(x[j])) for j in range(num_buttons))

def solve_input_text(inputs):
    results = []
    for raw in inputs:
        target_bits, buttons = parse_machine_line(raw)
        min_presses = solve_machine_ilp(target_bits, buttons)
        results.append(min_presses)
    return results

with open('input.txt','r') as f:
    inputs=[line.strip() for line in f.readlines()]
out = solve_input_text(inputs)
sum(out)


571

In [33]:
#part 2
import re
import pulp

def parse_line(line):
    target_match = re.search(r'\{([0-9,\s]+)\}', line)
    target = [int(x) for x in target_match.group(1).split(",")]
    k = len(target)
    groups = re.findall(r'\(([0-9,\s]*)\)', line)
    buttons = []
    for g in groups:
        inc = [0] * k
        if g.strip():
            for idx in map(int, g.split(",")):
                inc[idx] += 1
        buttons.append(inc)
    return target, buttons

def min_presses_joltage_ilp(target, buttons):
    m = len(buttons)
    k = len(target)
    prob = pulp.LpProblem("Min_Button_Presses", pulp.LpMinimize)
    x = [pulp.LpVariable(f"x{i}", lowBound=0, cat="Integer") for i in range(m)]
    # objective minimize presses
    prob += pulp.lpSum(x)
    # counter constraints
    for i in range(k):
        prob += pulp.lpSum(buttons[j][i] * x[j] for j in range(m)) == target[i]
    prob.solve(pulp.PULP_CBC_CMD())
    return int(pulp.value(prob.objective))

def solve_all_joltage_ilp(inputs):
    results = []
    for raw in inputs:
        target, buttons = parse_line(raw)
        ans = min_presses_joltage_ilp(target, buttons)
        results.append(ans)
    return results

with open('input.txt','r') as f:
    inputs=[line.strip() for line in f.readlines()]
out = solve_all_joltage_ilp(inputs)
sum(out)


20869